In [ ]:
%load_ext watermark


In [ ]:
from downstream import dstream
from hstrat import hstrat
from hstrat import _auxiliary_lib as hstrat_aux
import pandas as pd
import scipy

In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "2025-09-07-shortcut-naive-comparison"
teeplot_subdir


## Prep Data


In [ ]:
reconstruction_error_data = pd.read_csv("https://osf.io/wc8h5/download")
reconstruction_error_data.head()


In [ ]:
for group in reconstruction_error_data.groupby(["differentia_bitwidth", "surface_size", "fossil_interval"]):
    df = group[1]
    
    stat, p = scipy.stats.mannwhitneyu(
        df.iloc[1::2]["error_dropped_fossils"],  # shortcut
        df.iloc[::2]["error_dropped_fossils"],   # naive
        alternative="two-sided",
    )
    if p < 0.01:
        print(
            f"Significant difference in reconstruction error between shortcut and naive for differentia_bitwidth={group[0][0]}, surface_size={group[0][1]}, fossil_interval={group[0][2]}"
        )